# AI Capstone: Project 1
## *with a self-made dataset*
Author: 0816066 官澔恩

Data Source: [BBC News](https://www.bbc.com/news)

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_validate, ParameterGrid
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Data Preprocessing

In [2]:
data_dir = '../input/collection-of-bbc-news'
cats = ['business', 'entertainment-arts', 'science-environment', 'technology']

data = []
labels = []

for cat_idx, cat in enumerate(cats):
    cat_dir = f'{ data_dir }/{ cat }'
    for file_name in os.listdir(cat_dir):
        with open(f'{ cat_dir }/{ file_name }') as file:
            article = file.read()
            data.append(article)
            labels.append(cat_idx)

In [3]:
test_sizes = [0.2, 0.3]

# split data into training set and testing set based on the proportion of testing set
datasets = [ train_test_split(data, np.array(labels), test_size=test_size)
             for test_size in test_sizes ]

In [4]:
vectorizer = TfidfVectorizer(stop_words='english')

for idx, dataset in enumerate(datasets):
    X_train, X_test, y_train, y_test = dataset
    X_train = vectorizer.fit_transform(X_train).toarray()
    X_test = vectorizer.transform(X_test).toarray()
    datasets[idx] = X_train, X_test, y_train, y_test

# Models

In [5]:
# make a specified model with desired parameters
def get_model(model_type, params):
    if model_type == 'knn':
        return KNeighborsClassifier(**params)
    elif model_type == 'rf':
        return RandomForestClassifier(criterion='gini', **params)
    elif model_type == 'svm':
        return SVC(kernel='rbf', **params)
    elif model_type == 'mlp':
        return MLPClassifier(**params)
    else:
        return None

# Validation & Results

In [6]:
# display a confusion matrix and the classification report
def show_performance(y_true, y_pred):
    c_matrix = confusion_matrix(y_true, y_pred)
    c_table = pd.DataFrame(c_matrix)
    c_table.columns.name = 'truth\pred'
    display(c_table)
    
    report = classification_report(y_true, y_pred)
    print(report)

In [7]:
def show_cross_validate_report(res):
    report = pd.DataFrame({
        'fit_time': res['fit_time'],
        'score_time': res['score_time'],
        'test_score': res['test_score'],
    })
    display(report)

In [8]:
# train a model with 5-fold cross validation and validate the best model with the testing set
def train_model(model_type, param_grid, datasets):
    for test_size, dataset in zip(test_sizes, datasets):
        display(Markdown(f'### Test size: { test_size }'))
        X_train, X_test, y_train, y_test = dataset

        for params in param_grid:
            display(Markdown(f'#### { params }'))
            model = get_model(model_type, params)
            res = cross_validate(model, X_train, y_train, return_estimator=True)

            display(Markdown('#### Training Performance:'))
            show_cross_validate_report(res)

            best_model = res['estimator'][res['test_score'].argmax()]
            y_pred = best_model.predict(X_test)

            display(Markdown('#### Testing Performance:'))
            show_performance(y_test, y_pred)

## KNN

In [9]:
param_grid = ParameterGrid({
    'n_neighbors': [5, 10, 15]
})
train_model('knn', param_grid, datasets)

### Test size: 0.2

#### {'n_neighbors': 5}

#### Training Performance:

,fit_time,score_time,test_score
0,0.010232,0.014310,0.84375
1,0.012789,0.010397,0.78125
2,0.009912,0.010636,0.81250
3,0.011699,0.010509,0.90625
4,0.012049,0.010665,0.75000


#### Testing Performance:

...

#### Training Performance:

,fit_time,score_time,test_score
0,0.009740,0.008871,0.714286
1,0.009498,0.009081,0.821429
2,0.009432,0.008934,0.785714
3,0.009443,0.009038,0.714286
4,0.009315,0.008951,0.785714


#### Testing Performance:

truth\pred,0,1,2,3
0,19,0,0,3
1,0,16,0,0
2,0,0,9,0
3,3,1,1,8


              precision    recall  f1-score   support

           0       0.86      0.86      0.86        22
           1       0.94      1.00      0.97        16
           2       0.90      1.00      0.95         9
           3       0.73      0.62      0.67        13

    accuracy                           0.87        60
   macro avg       0.86      0.87      0.86        60
weighted avg       0.86      0.87      0.86        60



## Random Forest

In [10]:
param_grid = ParameterGrid({
    'min_samples_leaf': [1, 5, 10]
})
train_model('rf', param_grid, datasets)

### Test size: 0.2

#### {'min_samples_leaf': 1}

#### Training Performance:

,fit_time,score_time,test_score
0,0.276639,0.012397,0.78125
1,0.259322,0.011573,0.75000
2,0.260400,0.011800,0.93750
3,0.262405,0.011333,0.93750
4,0.265674,0.011716,0.87500


#### Testing Performance:

...

,fit_time,score_time,test_score
0,0.192890,0.011374,0.500000
1,0.190844,0.010893,0.642857
2,0.188658,0.010741,0.428571
3,0.190086,0.010827,0.642857
4,0.193840,0.012113,0.714286


#### Testing Performance:

truth\pred,0,1,2,3
0,0,7,3,12
1,0,13,2,1
2,0,0,9,0
3,0,2,4,7


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        22
           1       0.59      0.81      0.68        16
           2       0.50      1.00      0.67         9
           3       0.35      0.54      0.42        13

    accuracy                           0.48        60
   macro avg       0.36      0.59      0.44        60
weighted avg       0.31      0.48      0.37        60



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SVM

In [11]:
param_grid = ParameterGrid({
    'C': [1, 5, 10]
})
train_model('svm', param_grid, datasets)

### Test size: 0.2

#### {'C': 1}

#### Training Performance:

,fit_time,score_time,test_score
0,0.132141,0.079984,0.87500
1,0.132724,0.078518,0.81250
2,0.135332,0.078091,0.84375
3,0.131445,0.083116,0.87500
4,0.135578,0.084623,0.75000


#### Testing Performance:

...

#### Training Performance:

,fit_time,score_time,test_score
0,0.086067,0.027058,0.750000
1,0.088209,0.028067,0.857143
2,0.084602,0.027769,0.750000
3,0.085048,0.026573,0.714286
4,0.084978,0.027117,0.821429


#### Testing Performance:

truth\pred,0,1,2,3
0,14,1,1,6
1,0,16,0,0
2,0,0,9,0
3,0,1,2,10


              precision    recall  f1-score   support

           0       1.00      0.64      0.78        22
           1       0.89      1.00      0.94        16
           2       0.75      1.00      0.86         9
           3       0.62      0.77      0.69        13

    accuracy                           0.82        60
   macro avg       0.82      0.85      0.82        60
weighted avg       0.85      0.82      0.81        60



## MLP

In [12]:
param_grid = ParameterGrid({
    'hidden_layer_sizes': [256, 512, 1024]
})
train_model('mlp', param_grid, datasets)

### Test size: 0.2

#### {'hidden_layer_sizes': 256}

#### Training Performance:

,fit_time,score_time,test_score
0,11.662338,0.007427,0.87500
1,12.324114,0.007385,0.93750
2,11.536946,0.007498,0.87500
3,11.541674,0.007352,0.90625
4,11.544497,0.007532,0.90625


#### Testing Performance:

...

#### Training Performance:

,fit_time,score_time,test_score
0,20.549937,0.025915,0.857143
1,22.105150,0.026341,0.857143
2,22.933676,0.026634,0.750000
3,22.570739,0.026642,0.892857
4,22.303413,0.049539,0.964286


#### Testing Performance:

truth\pred,0,1,2,3
0,16,1,1,4
1,0,16,0,0
2,0,0,9,0
3,0,1,1,11


              precision    recall  f1-score   support

           0       1.00      0.73      0.84        22
           1       0.89      1.00      0.94        16
           2       0.82      1.00      0.90         9
           3       0.73      0.85      0.79        13

    accuracy                           0.87        60
   macro avg       0.86      0.89      0.87        60
weighted avg       0.89      0.87      0.86        60

